In [1]:
pip install tensorflow==0.12.0

ERROR: Could not find a version that satisfies the requirement tensorflow==0.12.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==0.12.0
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install tensorflow_text
!pip install tensorflow_hub

  Using cached MarkupSafe-2.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import tensorflow_text as text
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import os
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences

from collections import Counter
from keras.models import Sequential 
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

from keras import optimizers
from keras.backend import binary_crossentropy
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional

2022-12-01 18:41:04.447723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 18:41:04.569980: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-01 18:41:04.573758: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-01 18:41:04.573774: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [4]:
df= pd.read_csv("tweet_nlp_data_cleaned.csv")
df

,text,label,clean_tweet,text_normalization,word_tokenize,lemmatize_words
0,@BookstAnna Uncover the truth.\nStream #HOTD t...,HOTD,uncover the truth stream tonight at 9pm on rep...,uncover truth stream tonight 9pm reply,"['reply', 'truth', 'stream', 'tonight', 'uncov...","['reply', 'truth', 'stream', 'tonight', 'uncov..."
1,RT @arishatistic: the kiss we deserved\n\n#hot...,HOTD,the kiss we deserved,kiss deserved,"['kiss', 'deserved']","['kiss', 'deserved']"
2,RT @caffrey05: Ya quiero que se muera Viserys ...,HOTD,I already want Viserys to die so that the real...,I already want Viserys die real chaos begin ho...,"['real', 'Viserys', 'begin', 'die', 'I', 'chao...","['real', 'Viserys', 'begin', 'die', 'I', 'chao..."
3,RT @HOTDsource: More of Milly Alcock behind th...,HOTD,more of milly alcock behind the scenes of epis...,milly alcock behind scenes episode 5 adorable,"['episode', 'adorable', 'alcock', 'milly', 'sc...","['episode', 'adorable', 'alcock', 'milly', 'sc..."
4,@mfredericks1x Duty above all. \nStream #HOTD ...,HOTD,duty above all stream tonight at 9pm on reply ...,duty stream tonight 9pm reply,"['duty', 'tonight', 'stream', 'reply']","['duty', 'tonight', 'stream', 'reply']"
...,...,...,...,...,...,...
53849,Amazon’s Delay for ‘The Rings of Power’ Review...,ROP,amazon's delay for 'the rings of power' review...,amazon's delay 'the rings power' reviews prime...,"['pa', 'initiative', 'exclusive', 'filter', 't...","['pa', 'initiative', 'exclusive', 'filter', 't..."
53850,The latest #TheRingsOfPower trailer has us the...,ROP,the latest trailer has us theorizing to the mo...,latest trailer us theorizing moon back! read t...,"['back', 'thoughts', 'latest', 'read', 'us', '...","['back', 'thought', 'latest', 'read', 'u', 'th..."
53851,The Lord of the Rings: The Rings of Power Revi...,ROP,the lord of the rings the rings of power revie...,lord rings rings power review fantastic world ...,"['lord', 'world', 'search', 'power', 'rings', ...","['lord', 'world', 'search', 'power', 'ring', '..."
53852,Half way through the first episode of Rings of...,ROP,half way through the first episode of rings of...,half way first episode rings power already lov...,"['loving', 'aaaahhh', 'already', 'half', 'powe...","['loving', 'aaaahhh', 'already', 'half', 'powe..."


In [5]:
df_hotd= df[df["label"]=='HOTD']
df_hotd.shape

(26927, 6)

In [6]:
df_rop= df[df["label"]=='ROP']
df_rop.shape

(26927, 6)

In [7]:
df = df.sample(frac=1, random_state=1).reset_index()
df.drop('index', inplace=True, axis=1)
df

,text,label,clean_tweet,text_normalization,word_tokenize,lemmatize_words
0,Amazon Studios has launched “The Lord of the R...,ROP,"amazon studios has launched ""the lord of the r...","amazon studios launched ""the lord rings rings ...","['hobbit', 'take', 'tolkien', 'j', 'place', 's...","['hobbit', 'take', 'tolkien', 'j', 'place', 's..."
1,Lenny Henry praises new Lord of the Rings seri...,ROP,lenny henry praises new lord of the rings seri...,lenny henry praises new lord rings series '21s...,"['century', 'update', 'praises', 'lenny', 'lor...","['century', 'update', 'praise', 'lenny', 'lord..."
2,RT @HOTDsource: The #HouseOfTheDragon Season 1...,HOTD,the season 1 dragons so far,season 1 dragons far,"['dragons', 'far', 'season']","['dragon', 'far', 'season']"
3,RT @FreeFolkMemes: #SerCriston out here commit...,HOTD,out here committing hate crimes,committing hate crimes,"['crimes', 'committing', 'hate']","['crime', 'committing', 'hate']"
4,RT @HBOMaxLA: Un regalo para ti: Fondos de pan...,HOTD,a gift for you on wallpapers which one are you...,gift wallpapers one going use?,"['wallpapers', 'use', 'gift', 'going', 'one']","['wallpaper', 'use', 'gift', 'going', 'one']"
...,...,...,...,...,...,...
53849,@DanFriedman81 Rings of Power is getting revie...,ROP,rings of power is getting review bombed at the...,rings power getting review bombed moment fanta...,"['getting', 'pleasantly', 'far', 'moment', 'sh...","['getting', 'pleasantly', 'far', 'moment', 'sh..."
53850,"Surprise surprise, I really like the rings of ...",ROP,surprise surprise i really like the rings of p...,surprise surprise really like rings power far,"['like', 'surprise', 'really', 'power', 'rings...","['like', 'surprise', 'really', 'power', 'ring'..."
53851,RT @oochotd: The world was theirs #HouseOfTheD...,HOTD,the world was theirs,world,['world'],['world']
53852,Is House Of The Dragon based on source materia...,HOTD,is house of the dragon based on source materia...,house dragon based source material another one...,"['dragon', 'showrunners', 'stuff', 'based', 'u...","['dragon', 'showrunners', 'stuff', 'based', 'u..."


In [17]:
def counter_word(text):
  count= Counter()
  for i in text.values:
    for word in i.split():
      count[word]+=1
  return count

In [55]:
def split_modeling(df):
    df=df.astype(str)
    text= df.text_normalization
    counter= counter_word(text)

    num_words= len(counter)
    df.rename(columns = {'label':'Category', 'text_normalization':'Message'}, inplace = True)
    df['HOTD']= df['Category'].apply(lambda x: 1 if x== 'HOTD' else 0)

    #Max number of words in a sequence
    max_lenght= 50 #We are going to have the same sequence lenght for each 

    y= df.HOTD

    X_train, X_test, y_train, y_test= train_test_split(df['Message'], df['HOTD'], stratify=df['HOTD'])
    tokenizer= Tokenizer(num_words= num_words)
    tokenizer.fit_on_texts(X_train)
    word_index= tokenizer.word_index
    X_sequences= tokenizer.texts_to_sequences(X_train)

    train_padded= pad_sequences(X_sequences, 
                           maxlen=max_lenght, 
                           padding='post', 
                           truncating='post')

    test_sequences= tokenizer.texts_to_sequences(X_test)
    test_padded= pad_sequences(test_sequences, 
                           maxlen=max_lenght, 
                           padding='post', 
                           truncating='post')

    lb= LabelEncoder()
    y= lb.fit_transform(y_train)
    y_2= lb.fit_transform(y_test)

    return train_padded, test_padded, y_train, y_test, num_words, word_index, X_sequences, test_sequences

In [34]:
def model_lstm(train_padded, test_padded, y_train, y_test, num_words):
    max_lenght= 50
    model= Sequential()

    model.add(Embedding(num_words, 16, input_length=max_lenght))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(128, dropout=0.1, return_sequences=False)) ##32, 250
    model.add(Dropout(0.2))
    model.add(Dense(1, activation="sigmoid"))

    optimizer= Adam()

    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])


    model.fit(train_padded, y_train, epochs=10, validation_data=(test_padded, y_test),)
    return model

In [56]:
train_padded, test_padded, y_train, y_test, num_words, word_index, X_sequences, test_sequences= split_modeling(df)

In [36]:
y_train

3819     0
2558     0
49808    1
27827    0
35132    0
        ..
12       1
17860    0
11203    0
26659    1
9257     0
Name: HOTD, Length: 40390, dtype: int64

In [37]:
model= model_lstm(train_padded, test_padded, y_train, y_test, num_words)

Epoch 1/10
1263/1263 [==============================] - 98s 76ms/step - loss: 0.4428 - accuracy: 0.7775 - val_loss: 0.4640 - val_accuracy: 0.7659
Epoch 2/10
1263/1263 [==============================] - 95s 75ms/step - loss: 0.4085 - accuracy: 0.8176 - val_loss: 0.3586 - val_accuracy: 0.8586
Epoch 3/10
1263/1263 [==============================] - 96s 76ms/step - loss: 0.3461 - accuracy: 0.8662 - val_loss: 0.3311 - val_accuracy: 0.8755
Epoch 4/10
1263/1263 [==============================] - 95s 76ms/step - loss: 0.3160 - accuracy: 0.8852 - val_loss: 0.3065 - val_accuracy: 0.8936
Epoch 5/10
1263/1263 [==============================] - 99s 78ms/step - loss: 0.2803 - accuracy: 0.9100 - val_loss: 0.2767 - val_accuracy: 0.9192
Epoch 6/10
1263/1263 [==============================] - 95s 76ms/step - loss: 0.2523 - accuracy: 0.9292 - val_loss: 0.2731 - val_accuracy: 0.9210
Epoch 7/10
1263/1263 [==============================] - 95s 75ms/step - loss: 0.2563 - accuracy: 0.9250 - val_loss: 0.2918 -

In [38]:
score = model.evaluate(test_padded, y_test)
model.save("results/my_model_LSTM_classifier.h5")

421/421 [==============================] - 8s 18ms/step - loss: 0.3581 - accuracy: 0.8867


## Testing

In [40]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3581242561340332
Test accuracy: 0.8867350220680237


In [62]:
y_pred = model.predict(test_padded)

421/421 [==============================] - 8s 18ms/step


In [63]:
y_pred=(y_pred>0.5)
y_pred

array([[ True],
       [False],
       [ True],
       ...,
       [False],
       [False],
       [False]])

In [57]:
reverse_word_index= dict([(value,key) for (key,value) in word_index.items()])

def decode (text):
  return " ".join([reverse_word_index.get(i,"?") for i in text])

In [65]:
decode(X_sequences[7])

'tuesday watch fifth episode house dragon yet guess loosing interest least captain ed mercer wo hy good laugh'

In [66]:
y_pred[7]

array([ True])

## Trying with new data

In [67]:
def word_count(text):
  count= Counter()
  for i in text:
    for word in i.split():
      count[word]+=1
  return count
  
def input_cleaner(text):
    counter= word_count(text)
    max_lenght= 80
    num_words= len(counter)
    #count= 80

    tokenizer= Tokenizer(num_words= num_words)
    tokenizer.fit_on_texts(text)
    word_index= tokenizer.word_index
    text_sequence = tokenizer.texts_to_sequences(text)

    
    text_padded= pad_sequences(text_sequence, 
                           maxlen=80, 
                           padding='post', 
                           truncating='post')
    return text_padded

In [72]:
text = ['ring of the power is the best survive']
text_padded= input_cleaner(text)
text_padded

array([[2, 3, 1, 4, 5, 1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [73]:
y_pred = model.predict(text_padded)
y_pred=(y_pred>0.5)
y_pred

1/1 [==============================] - 0s 29ms/step


array([[False]])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d578b49a-f433-4f75-ac7b-2845145b8de8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>